In [1]:
import os
from PIL import Image
import numpy as np
from numpy.linalg import norm, pinv
np.set_printoptions(precision=12, suppress=True)

In [2]:
consider_distort = False # flag whether to consider distortion
zoomed_coords = True # whether to divide coordinates by 5 or not (zoomed coords or not)
SCALE_FAC = 4.0 # Scale factor of coordinates

In [3]:
folder_name = 'data/star_coords/2016nov-11_txt//' # folder with coords files

In [4]:
# 20160218
# fnames = [
#     '20160218-201934-437.txt',
#     '20160218-211934-484.txt'
# ]

# 20160219
# fnames = [
#     '20160219-001934-484.txt',
#     '20160219-021934-593.txt',
#     '20160219-041934-593.txt'
# ]

# 20160311
# fnames = [
#     '20160311-020811-125.txt',
#     '20160311-030811-125.txt'
# ]

# 20161122 Nov-11
fnames = [
#     '20161122-221517-375.txt',
    "20161122-191517-359.txt"
]

# 20161121
# fnames = [
# #     '20161121-220921-250.txt',
    
# ]

In [5]:
if consider_distort:
    print "consider_distort is True!"

In [6]:
coords_list = []
for fname in fnames:
    piece = np.loadtxt(folder_name + os.sep + fname)
    coords_list.append(piece)

coords = np.vstack(coords_list)
print 'Zoomed In Star coordinates pairs:\n', coords, '\n'

if zoomed_coords:
    coords /= float(SCALE_FAC)
    coords = coords.round()
    print 'Normal Star coordinates pairs:\n', coords

Zoomed In Star coordinates pairs:
[[  5734.   6266.   6711.   6411.]
 [  6490.   6723.   7468.   6866.]
 [  5934.   7360.   6910.   7507.]
 [  6209.   7975.   7190.   8129.]
 [  9774.   8478.  10880.   8711.]
 [ 10446.   6163.  11559.   6314.]
 [ 10446.   3421.  11514.   3477.]
 [  8961.   1184.   9917.   1233.]
 [  2774.    834.   3744.   1098.]
 [  2590.   4247.   3603.   4430.]
 [  3413.   3882.   4398.   4059.]] 

Normal Star coordinates pairs:
[[ 1434.  1566.  1678.  1603.]
 [ 1622.  1681.  1867.  1716.]
 [ 1484.  1840.  1728.  1877.]
 [ 1552.  1994.  1798.  2032.]
 [ 2444.  2120.  2720.  2178.]
 [ 2612.  1541.  2890.  1578.]
 [ 2612.   855.  2878.   869.]
 [ 2240.   296.  2479.   308.]
 [  694.   208.   936.   274.]
 [  648.  1062.   901.  1108.]
 [  853.   970.  1100.  1015.]]


In [7]:
lX = coords[:, 0] # leftX coordinates
lY = coords[:, 1] # leftY coordinates
rX = coords[:, 2] # rightX coordinates
rY = coords[:, 3] # rightY coordinates

N = coords.shape[0] # number of pairs of points
M = coords.shape[1] # lX, lY, rX, rY == 4
print 'Number of Star coordinates pairs:', N

Number of Star coordinates pairs: 11


In [8]:
xi = np.zeros(2*N)

# First, calculate E,F
num_coeffs = 2
if consider_distort:
    num_coeffs = 8

z = np.zeros(num_coeffs)
arr = np.zeros((2*N, num_coeffs)) # matrix A
print 'Number of coefficients:', num_coeffs

Number of coefficients: 2


In [9]:
for i in xrange(N): # fill the xi vector
    xi[2*i] = rX[i] - lX[i]
    xi[2*i + 1] = rY[i] - lY[i]

In [10]:
if consider_distort:
    im = Image.open(folder_name + 'mod_'+ fnames[0][:-4] +'-1.png')
    x_c = im.width / 2.0
    y_c = im.height / 2.0
    for i in xrange(N): # fill the A matrix
        dist_l = (lX[i]-x_c)**2 + (lY[i]-y_c)**2
        dist_r = (rX[i]-x_c)**2 + (rY[i]-y_c)**2

        zx1 = (lX[i] - x_c) * dist_l
        zx2 = (rX[i] - x_c) * dist_r
        arr[2*i] = [lX[i], lY[i], 0, 0, 1, 0, -zx1, zx2]

        zy1 = (lY[i] - y_c) * dist_l
        zy2 = (rY[i] - y_c) * dist_r
        arr[2*i + 1] = [0, 0, lX[i], lY[i], 0, 1, -zy1, zy2]

else:
    for i in xrange(N): # fill the A matrix
        arr[2*i] = [1, 0]

        arr[2*i + 1] = [0, 1]

In [11]:
np.set_printoptions(precision=2, suppress=True)
print 'A:\n', arr, '\n'
print 'xi:\n', xi
np.set_printoptions(precision=12, suppress=True)

A:
[[ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]] 

xi:
[ 244.   37.  245.   35.  244.   37.  246.   38.  276.   58.  278.   37.
  266.   14.  239.   12.  242.   66.  253.   46.  247.   45.]


In [12]:
p_arr = pinv(arr)
z = np.dot(p_arr, xi)
ef = z
e = z[0]
f = z[1]
print 'Affine coefficients EF: \n', z

Affine coefficients EF: 
[ 252.727272727273   38.636363636364]


In [13]:
# Second, calculate ABCD
num_coeffs = 4

z = np.zeros(num_coeffs)
arr = np.zeros((2*N, num_coeffs)) # matrix A
print 'Number of coefficients:', num_coeffs

Number of coefficients: 4


In [14]:
for i in xrange(N): # fill the xi vector
    xi[2*i] = rX[i] - e
    xi[2*i + 1] = rY[i] - f

In [15]:
if consider_distort:
    im = Image.open(folder_name + 'mod_'+ fnames[0][:-4] +'-1.png')
    x_c = im.width / 2.0
    y_c = im.height / 2.0
    for i in xrange(N): # fill the A matrix
        dist_l = (lX[i]-x_c)**2 + (lY[i]-y_c)**2
        dist_r = (rX[i]-x_c)**2 + (rY[i]-y_c)**2

        zx1 = (lX[i] - x_c) * dist_l
        zx2 = (rX[i] - x_c) * dist_r
        arr[2*i] = [lX[i], lY[i], 0, 0, 1, 0, -zx1, zx2]

        zy1 = (lY[i] - y_c) * dist_l
        zy2 = (rY[i] - y_c) * dist_r
        arr[2*i + 1] = [0, 0, lX[i], lY[i], 0, 1, -zy1, zy2]

else:
    for i in xrange(N): # fill the A matrix
        arr[2*i] = [lX[i], lY[i], 0, 0]

        arr[2*i + 1] = [0, 0, lX[i], lY[i]]

In [16]:
np.set_printoptions(precision=2, suppress=True)
print 'A:\n', arr, '\n'
print 'xi:\n', xi
np.set_printoptions(precision=12, suppress=True)

A:
[[ 1434.  1566.     0.     0.]
 [    0.     0.  1434.  1566.]
 [ 1622.  1681.     0.     0.]
 [    0.     0.  1622.  1681.]
 [ 1484.  1840.     0.     0.]
 [    0.     0.  1484.  1840.]
 [ 1552.  1994.     0.     0.]
 [    0.     0.  1552.  1994.]
 [ 2444.  2120.     0.     0.]
 [    0.     0.  2444.  2120.]
 [ 2612.  1541.     0.     0.]
 [    0.     0.  2612.  1541.]
 [ 2612.   855.     0.     0.]
 [    0.     0.  2612.   855.]
 [ 2240.   296.     0.     0.]
 [    0.     0.  2240.   296.]
 [  694.   208.     0.     0.]
 [    0.     0.   694.   208.]
 [  648.  1062.     0.     0.]
 [    0.     0.   648.  1062.]
 [  853.   970.     0.     0.]
 [    0.     0.   853.   970.]] 

xi:
[ 1425.27  1564.36  1614.27  1677.36  1475.27  1838.36  1545.27  1993.36
  2467.27  2139.36  2637.27  1539.36  2625.27   830.36  2226.27   269.36
   683.27   235.36   648.27  1069.36   847.27   976.36]


In [17]:
p_arr = pinv(arr)
z = np.dot(p_arr, xi)
abcd = z
print 'Affine coefficients ABCD: \n', z

Affine coefficients ABCD: 
[ 1.003599959942 -0.002516674026 -0.009362645534  1.010974300678]


In [18]:
z = np.hstack([abcd, ef])
print 'Affine coefficients: \n', z

Affine coefficients: 
[   1.003599959942   -0.002516674026   -0.009362645534    1.010974300678
  252.727272727273   38.636363636364]


In [19]:
sigma = 2 # 2 pix error
I = np.eye(2*N)

print 'cond(A): ', np.linalg.cond(arr)
proba = (2*N - num_coeffs) / norm( np.dot(I - np.dot(arr, p_arr), xi) )**2
sig_theory = (1 / proba)**0.5

print 'Probability:', proba
print 'Sigma theory:', sig_theory
print 'Sigma practice', sigma

cond(A):  3.89908880211
Probability: 0.00503380919554
Sigma theory: 14.0945633226
Sigma practice 2


In [20]:
res_fname = 'results/2016nov-11//'

if consider_distort:
    res_fname += 'affdis_coeffs_shots'
else:
    res_fname += 'affine_coeffs_shots'

res_fname += str(len(fnames)) + '_' + fnames[0]

In [21]:
np.savetxt(res_fname, z, fmt="%.10f")
print 'Coeffs are saved to:', res_fname

Coeffs are saved to: results/2016nov-11//affine_coeffs_shots1_20161122-191517-359.txt
